In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
import urllib.request

url = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
page = urllib.request.urlopen(url) 

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page, "lxml")

In [7]:
soup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   Demographics of Toronto neighbourhoods - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XsEgMApAMOIAAANxgD8AAAAI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Demographics_of_Toronto_neighbourhoods","wgTitle":"Demographics of Toronto neighbourhoods","wgCurRevisionId":955167726,"wgRevisionId":955167726,"wgArticleId":38958143,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with obsolete information from April 2013","All Wikipedia articles in

In [8]:
all_tables=soup.find_all("table")
all_tables

[<table class="box-Update plainlinks metadata ambox ambox-content ambox-Update" role="presentation"><tbody><tr><td class="mbox-image"><div style="width:52px"><img alt="Ambox current red.svg" data-file-height="290" data-file-width="360" decoding="async" height="34" src="//upload.wikimedia.org/wikipedia/commons/thumb/9/98/Ambox_current_red.svg/42px-Ambox_current_red.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/9/98/Ambox_current_red.svg/63px-Ambox_current_red.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/9/98/Ambox_current_red.svg/84px-Ambox_current_red.svg.png 2x" width="42"/></div></td><td class="mbox-text"><div class="mbox-text-span">This article's <b><a href="/wiki/Wikipedia:Accuracy_dispute" title="Wikipedia:Accuracy dispute">factual accuracy</a> may be compromised due to out-of-date information</b>.<span class="hide-when-compact"> Please update this article to reflect recent events or newly available information.</span> <small class="date-container

In [9]:
toronto_table=soup.find('table', class_='wikitable sortable')
toronto_table

<table border="1" cellpadding="5" cellspacing="0" class="wikitable sortable">
<tbody><tr>
<th width="20%">Name
</th>
<th width="5%">FM
</th>
<th width="20%">Census Tracts
</th>
<th width="5%">Population
</th>
<th width="5%">Land area (km2)
</th>
<th width="5%">Density (people/km2)
</th>
<th width="5%">% Change in Population since 2001
</th>
<th width="5%">Average Income
</th>
<th width="5%">Transit Commuting %
</th>
<th width="5%">% Renters
</th>
<th width="10%">Second most common language (after English) by name
</th>
<th width="10%">Second most common language (after English) by percentage
</th>
<th width="10%">Map
</th></tr>
<tr>
<td><b>Toronto <a class="mw-redirect" href="/wiki/Census_metropolitan_area" title="Census metropolitan area">CMA</a> Average</b>
</td>
<td>
</td>
<td>All
</td>
<td><b>5,113,149</b>
</td>
<td><b>5903.63</b>
</td>
<td><b>866</b>
</td>
<td><b>9.0</b>
</td>
<td><b>40,704</b>
</td>
<td><b>10.6</b>
</td>
<td><b>11.4</b>
</td>
<td>
</td>
<td>
</td>
<td>
</td></tr>

In [23]:
population=[3]
neighborhood=[0]
density=[5]


for row in toronto_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==13:
        a=cells[0].find(text=True)
        a=a[:]
        neighborhood.append(a)
        b=cells[3].find(text=True)
        b=b[:-1]
        population.append(b)
        c=cells[5].find(text=True)
        c=c[:-1]
        density.append(c)

In [24]:
df = pd.DataFrame(neighborhood,columns=['Neighborhood'])

In [25]:
df['Population'] = population
df['density'] = density
df

,Neighborhood,Population,density
0,0,3,5
1,Toronto,"5,113,14",86
2,Agincourt,"44,577",3580
3,Alderwood,"11,656",2360
4,Alexandra Park,"4,355","13,609"
5,Allenby,"2,513",4333
6,Amesbury,"17,318","4,934"
7,Armour Heights,"4,384",1914
8,Banbury,"6,641",2442
9,Bathurst Manor,"14,945",3187


In [28]:
df.drop(df.index[0],inplace=True)

,Neighborhood,Population,density
2,Agincourt,"44,577",3580
3,Alderwood,"11,656",2360
4,Alexandra Park,"4,355","13,609"
5,Allenby,"2,513",4333
6,Amesbury,"17,318","4,934"
7,Armour Heights,"4,384",1914
8,Banbury,"6,641",2442
9,Bathurst Manor,"14,945",3187
10,Bay Street Corridor,"4,787","43,518"
11,Bayview Village,"12,280","2,966"


In [33]:
df.reset_index(inplace=True)

,index,Neighborhood,Population,density
0,2,Agincourt,"44,577",3580
1,3,Alderwood,"11,656",2360
2,4,Alexandra Park,"4,355","13,609"
3,5,Allenby,"2,513",4333
4,6,Amesbury,"17,318","4,934"
5,7,Armour Heights,"4,384",1914
6,8,Banbury,"6,641",2442
7,9,Bathurst Manor,"14,945",3187
8,10,Bay Street Corridor,"4,787","43,518"
9,11,Bayview Village,"12,280","2,966"


In [34]:
df.drop('index',axis=1,inplace=True)

In [35]:
df

,Neighborhood,Population,density
0,Agincourt,"44,577",3580
1,Alderwood,"11,656",2360
2,Alexandra Park,"4,355","13,609"
3,Allenby,"2,513",4333
4,Amesbury,"17,318","4,934"
5,Armour Heights,"4,384",1914
6,Banbury,"6,641",2442
7,Bathurst Manor,"14,945",3187
8,Bay Street Corridor,"4,787","43,518"
9,Bayview Village,"12,280","2,966"


In [117]:
url2 = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page2 = urllib.request.urlopen(url2)
soup2 = BeautifulSoup(page2, "lxml")
toronto_table2=soup2.find('table', class_='wikitable sortable')
toronto_table2

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>
</td></tr>
<tr>
<td>M3B
</td>
<td>North York
</td>
<td>Don Mills
</td></tr>
<tr>
<td>M4B
</td>
<td>East Y

In [118]:
postal_code=[]
borough=[]
neighborhood=[]


for row in toronto_table2.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        a=cells[0].find(text=True)
        a=a[:-1]
        postal_code.append(a)
        b=cells[1].find(text=True)
        b=b[:-1]
        borough.append(b)
        c=cells[2].find(text=True)
        c=c[:-1]
        neighborhood.append(c)

df2 = pd.DataFrame(postal_code,columns=['PostalCode'])
df2['Borough'] = borough
df2['Neighborhood'] = neighborhood

In [119]:
df2

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Malvern, Rouge"


In [120]:
df2.drop(df2[df2['Borough']=='Not assigned'].index,inplace=True)

for i in range(103):
    if(df2.iloc[i,2]=='Not assigned'):
        df2.iloc[i,2] = df2.iloc[i,1]
        
df2.reset_index(drop=True, inplace=True)

In [121]:
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [122]:
df.shape

(174, 4)

In [66]:
area=[]
for i in range(174):
    area.append(0.0)
df['Area']=area

In [67]:
df

,Neighborhood,Population,density,Area
0,Agincourt,"44,577",3580,0.0
1,Alderwood,"11,656",2360,0.0
2,Alexandra Park,"4,355","13,609",0.0
3,Allenby,"2,513",4333,0.0
4,Amesbury,"17,318","4,934",0.0
5,Armour Heights,"4,384",1914,0.0
6,Banbury,"6,641",2442,0.0
7,Bathurst Manor,"14,945",3187,0.0
8,Bay Street Corridor,"4,787","43,518",0.0
9,Bayview Village,"12,280","2,966",0.0


In [71]:
for i in range(174):
    df.iloc[i,3]=float(df.iloc[i,1].replace(',',''))/float(df.iloc[i,2].replace(',',''))

In [72]:
df

,Neighborhood,Population,density,Area
0,Agincourt,"44,577",3580,12.451676
1,Alderwood,"11,656",2360,4.938983
2,Alexandra Park,"4,355","13,609",0.320009
3,Allenby,"2,513",4333,0.579968
4,Amesbury,"17,318","4,934",3.509931
5,Armour Heights,"4,384",1914,2.290491
6,Banbury,"6,641",2442,2.719492
7,Bathurst Manor,"14,945",3187,4.689363
8,Bay Street Corridor,"4,787","43,518",0.110000
9,Bayview Village,"12,280","2,966",4.140256


In [123]:
population=[]
area=[]
density=[]

for i in range(103):
    population.append(0)
    area.append(0.0)
    density.append(0)
    
df2['Area'] = area
df2['Population'] = population
df2['Density'] = density

In [124]:
df2

,PostalCode,Borough,Neighborhood,Area,Population,Density
0,M3A,North York,Parkwoods,0.0,0,0
1,M4A,North York,Victoria Village,0.0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0.0,0,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",0.0,0,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.0,0,0
5,M9A,Etobicoke,Islington Avenue,0.0,0,0
6,M1B,Scarborough,"Malvern, Rouge",0.0,0,0
7,M3B,North York,Don Mills,0.0,0,0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0.0,0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0.0,0,0


In [125]:
dummy = df2['Neighborhood'].tolist()
len(dummy)

103

In [126]:
for i in range(174):
    for index in range(103):
        if(df.iloc[i,0] in dummy[index]):
            df2.iloc[index,4] = int(df2.iloc[index,4]) + int(df.iloc[i,1].replace(',','')) 
            df2.iloc[index,3] = float(df2.iloc[index,3]) + float(df.iloc[i,3]) 
            break
    

In [127]:
df2

,PostalCode,Borough,Neighborhood,Area,Population,Density
0,M3A,North York,Parkwoods,4.960366,26533,0
1,M4A,North York,Victoria Village,4.719546,17047,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.000000,0,0
5,M9A,Etobicoke,Islington Avenue,0.000000,0,0
6,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,0
7,M3B,North York,Don Mills,8.991165,21372,0
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0.000000,0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,0


In [128]:
for i in range(174):
    try:
        df2.iloc[i,5] = int(df2.iloc[i,4]/df2.iloc[i,3])
    except:
        pass

D:\Installations\Anaconda\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [129]:
df2

,PostalCode,Borough,Neighborhood,Area,Population,Density
0,M3A,North York,Parkwoods,4.960366,26533,5349
1,M4A,North York,Victoria Village,4.719546,17047,3612
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,11267
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,3280
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",0.000000,0,0
5,M9A,Etobicoke,Islington Avenue,0.000000,0,0
6,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,1783
7,M3B,North York,Don Mills,8.991165,21372,2376
8,M4B,East York,"Parkview Hill, Woodbine Gardens",0.000000,0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,15846


In [135]:
df2.shape

(63, 6)

In [136]:
df2 = df2[df2.Population!=0]

In [139]:
df2


,PostalCode,Borough,Neighborhood,Area,Population,Density
0,M3A,North York,Parkwoods,4.960366,26533,5349
1,M4A,North York,Victoria Village,4.719546,17047,3612
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,11267
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,3280
6,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,1783
7,M3B,North York,Don Mills,8.991165,21372,2376
9,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,15846
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",6.260228,13683,2185
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",14.400674,36470,2532
15,M5C,Downtown Toronto,St. James Town,0.230001,14666,63765


In [140]:
toronto_df = df2

In [141]:
df2 = pd.read_csv(r"C:\Users\Vijay\Downloads\GC.csv")

In [142]:
df2

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [143]:
Latitude=[]
Longitude=[]

for i in range(63):
    Latitude.append(0.0)
    Longitude.append(0.0)
    
toronto_df['Latitude']=Latitude
toronto_df['Longitude']=Longitude


In [144]:
toronto_df

,PostalCode,Borough,Neighborhood,Area,Population,Density,Latitude,Longitude
0,M3A,North York,Parkwoods,4.960366,26533,5349,0.0,0.0
1,M4A,North York,Victoria Village,4.719546,17047,3612,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,11267,0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,3280,0.0,0.0
6,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,1783,0.0,0.0
7,M3B,North York,Don Mills,8.991165,21372,2376,0.0,0.0
9,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,15846,0.0,0.0
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",6.260228,13683,2185,0.0,0.0
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",14.400674,36470,2532,0.0,0.0
15,M5C,Downtown Toronto,St. James Town,0.230001,14666,63765,0.0,0.0


In [145]:
toronto_df.reset_index()

,index,PostalCode,Borough,Neighborhood,Area,Population,Density,Latitude,Longitude
0,0,M3A,North York,Parkwoods,4.960366,26533,5349,0.0,0.0
1,1,M4A,North York,Victoria Village,4.719546,17047,3612,0.0,0.0
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,11267,0.0,0.0
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,3280,0.0,0.0
4,6,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,1783,0.0,0.0
5,7,M3B,North York,Don Mills,8.991165,21372,2376,0.0,0.0
6,9,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,15846,0.0,0.0
7,11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",6.260228,13683,2185,0.0,0.0
8,12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",14.400674,36470,2532,0.0,0.0
9,15,M5C,Downtown Toronto,St. James Town,0.230001,14666,63765,0.0,0.0


In [151]:
toronto_df.reset_index(drop=True,inplace=True)

In [152]:
toronto_df

,PostalCode,Borough,Neighborhood,Area,Population,Density,Latitude,Longitude
0,M3A,North York,Parkwoods,4.960366,26533,5349,0.0,0.0
1,M4A,North York,Victoria Village,4.719546,17047,3612,0.0,0.0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755,11267,0.0,0.0
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519,3280,0.0,0.0
4,M1B,Scarborough,"Malvern, Rouge",37.587676,67048,1783,0.0,0.0
5,M3B,North York,Don Mills,8.991165,21372,2376,0.0,0.0
6,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240,15846,0.0,0.0
7,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",6.260228,13683,2185,0.0,0.0
8,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",14.400674,36470,2532,0.0,0.0
9,M5C,Downtown Toronto,St. James Town,0.230001,14666,63765,0.0,0.0


In [159]:
for i in range(103):
    code = df2.iloc[i,0]
    lat = df2.iloc[i,1]
    lon = df2.iloc[i,2]
    index = toronto_df.index[toronto_df['PostalCode']==code].tolist()
    try:
        toronto_df.iloc[index[0],6] = lat
        toronto_df.iloc[index[0],7] = lon
    except:
        pass

In [160]:
toronto_df

,PostalCode,Borough,Neighborhood,Area,Population,Density,Latitude,Longitude
0,M3A,North York,Parkwoods,4.960366,26533.000000,5349,43.753259,-79.329656
1,M4A,North York,Victoria Village,4.719546,17047.000000,3612,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",2.196988,24755.000000,11267,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",5.339568,17519.000000,3280,43.718518,-79.464763
4,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1783,43.806686,-79.194353
5,M3B,North York,Don Mills,8.991165,21372.000000,2376,43.745906,-79.352188
6,M5B,Downtown Toronto,"Garden District, Ryerson",0.520005,8240.000000,15846,43.657162,-79.378937
7,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",6.260228,13683.000000,2185,43.650943,-79.554724
8,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,2532,43.784535,-79.160497
9,M5C,Downtown Toronto,St. James Town,0.230001,14666.000000,63765,43.651494,-79.375418
